main.py


In [ ]:
import cv2
import config
from game_manager import GameManager

# --- SETUP ---
face_cascade = cv2.CascadeClassifier(config.FACE_CASCADE_PATH)
video_capture = cv2.VideoCapture(0)
video_capture.set(cv2.CAP_PROP_FRAME_WIDTH, config.SCREEN_WIDTH)
video_capture.set(cv2.CAP_PROP_FRAME_HEIGHT, config.SCREEN_HEIGHT)

game_manager = GameManager()

# --- MAIN GAME LOOP ---
while True:
    ret, frame = video_capture.read()
    if not ret: break

    frame = cv2.flip(frame, 1)
    
    scale = config.DETECTION_SCALE_FACTOR
    small_frame = cv2.resize(frame, (frame.shape[1] // scale, frame.shape[0] // scale))
    gray = cv2.cvtColor(small_frame, cv2.COLOR_BGR2GRAY)
    
    faces = face_cascade.detectMultiScale(gray, 1.1, 5)

    face_center_x = None
    if len(faces) > 0:
        (x, y, w, h) = [v * scale for v in faces[0]]
        face_center_x = x + w // 2

    # --- DELEGATE ALL GAME LOGIC TO THE MANAGER ---
    game_manager.update(frame, face_center_x)

    cv2.imshow('Face Pong', frame)

    # --- KEYBOARD INPUT ---
    key = cv2.waitKey(1) & 0xFF
    if key == ord('q'):
        break
    if key == ord('r') and game_manager.game_state == "game_over":
        game_manager.reset_game()

# --- CLEANUP ---
video_capture.release()
cv2.destroyAllWindows()


config.py

In [ ]:
import cv2

# Screen dimensions
SCREEN_WIDTH = 1280
SCREEN_HEIGHT = 720

# Performance settings
DETECTION_SCALE_FACTOR = 4   
FACE_CASCADE_PATH = "assets/haarcascade_frontalface_alt.xml"
FACE_CONTROL_ZONE_PERCENT = 0.5 

# --- Game Rule Settings ---
INITIAL_LIVES = 3
POINTS_PER_HIT = 10
HITS_PER_LEVEL = 2 
BALL_SPEED_INCREASE = 2.0

# --- UI Settings ---
UI_FONT = cv2.FONT_HERSHEY_DUPLEX
UI_FONT_SCALE = 1.0
UI_FONT_THICKNESS = 2
UI_COLOR = (0, 255, 100) # Vibrant Green
UI_OUTLINE_COLOR = (0, 0, 0) # Black outline for text
SCORE_POSITION = (30, 50)    # Top-left corner
LIVES_POSITION = (SCREEN_WIDTH - 180, 50)  # Top-right corner
LEVEL_POSITION = (SCREEN_WIDTH // 2 - 80, 50)  # Top-center
GAME_OVER_POSITION = (SCREEN_WIDTH // 2 - 250, SCREEN_HEIGHT // 2)  # Centered
POWERUP_TIMER_POSITION = (SCREEN_WIDTH // 2 - 200, SCREEN_HEIGHT - 20)  # Bottom-center

# --- Paddle Settings ---
PADDLE_WIDTH = 200
PADDLE_HEIGHT = 25
PADDLE_SMOOTHING = 0.12     
PADDLE_BOUNCE_INTENSITY = 8.0   
PADDLE_WIDE_FACTOR = 1.5
PADDLE_SHRINK_FACTOR = 0.5
PADDLE_COLOR_TOP = (255, 100, 0)
PADDLE_COLOR_BOTTOM = (200, 50, 0)
PADDLE_HIGHLIGHT_COLOR = (255, 255, 255)

# --- Ball Settings ---
BALL_RADIUS = 18 
BALL_IMAGE_PATH = "assets/ball_3d.png"
BALL_INITIAL_VX = 7.2
BALL_INITIAL_VY = -7.2
BALL_SLOW_FACTOR = 0.5    #for power-up
BALL_FAST_FACTOR = 1.5    #for power-up

# --- Power-Up Settings ---
POWERUP_SIZE = 40
POWERUP_SPAWN_CHANCE = 0.3   # 30% chance on paddle hit
POWERUP_SPEED_Y = 4
POWERUP_SPEED_X_MAX = 3
POWERUP_DURATION_SECONDS = 8.0
POWERUP_MAX_ON_SCREEN = {1: 0, 2: 1, 5: 2, 8: 3}
POWERUP_UNLOCK_LEVELS = {"wide_paddle": 2, "shrink_paddle": 3, "slow_ball": 4, "fast_ball": 5, "extra_life": 6}
POWERUP_TYPES = {
    "wide_paddle": {"image": "assets/powerup_wide.png"},
    "extra_life": {"image": "assets/powerup_life.png"},
    "slow_ball": {"image": "assets/powerup_slow.png"},
    "shrink_paddle": {"image": "assets/powerup_shrink.png"},
    "fast_ball": {"image": "assets/powerup_fast.png"}
}

game_manager.py


In [ ]:
import cv2
import config
import random
import time
from game_objects import Paddle, Ball, PowerUp

class GameManager:
    def __init__(self):
        self.paddle = Paddle()
        self.ball = Ball()
        self.powerups = []
        self.active_effects = {}
        self.reset_game()

    def reset_game(self):
        self.score = 0
        self.lives = config.INITIAL_LIVES
        self.level = 1
        self.hits_this_level = 0
        self.game_state = "playing"
        self.powerups = []
        self.deactivate_all_effects()
        self.ball.reset(level=1)

    def level_up(self):
        self.level += 1
        self.hits_this_level = 0
        speed_increase = config.BALL_SPEED_INCREASE
        max_speed = 15
        self.ball.vx = min(abs(self.ball.vx) + speed_increase, max_speed) * (1 if self.ball.vx > 0 else -1)
        self.ball.vy = min(abs(self.ball.vy) + speed_increase, max_speed) * (1 if self.ball.vy > 0 else -1)

    def update(self, frame, face_center_x):
        if self.game_state == "playing":
            if face_center_x: self.paddle.update(face_center_x)
            self.ball.move()
            self.update_powerups()
            self.update_effects()
            self.check_collisions()
        elif self.game_state == "game_over":
     
                 # Draw the main "GAME OVER" text
            self.draw_text_with_outline(frame, "GAME OVER", config.GAME_OVER_POSITION, config.UI_FONT, 2.5, config.UI_COLOR, 5)
            
            # Define the restart text and get its size
            restart_text = "Press 'R' to Restart"
            text_size = cv2.getTextSize(restart_text, config.UI_FONT, 1.0, 3)[0]
            
            # Calculate the centered position for the restart text
            restart_pos_x = (config.SCREEN_WIDTH - text_size[0]) // 2
            restart_pos_y = config.GAME_OVER_POSITION[1] + 60
            
            # Draw the restart text
            self.draw_text_with_outline(frame, restart_text, (restart_pos_x, restart_pos_y), config.UI_FONT, 1.0, config.UI_COLOR, 3)
        
        for p in self.powerups: p.draw(frame)
        self.paddle.draw(frame)
        self.ball.draw(frame)
        self.draw_ui(frame)

    def draw_text_with_outline(self, frame, text, pos, font, scale, color, thickness):
        # Draw the black outline
        cv2.putText(frame, text, (pos[0]+2, pos[1]+2), font, scale, config.UI_OUTLINE_COLOR, thickness + 2)
        # Draw the main text
        cv2.putText(frame, text, pos, font, scale, color, thickness)

    def draw_ui(self, frame):
        score_text = f"Score: {self.score}"
        lives_text = f"Lives: {self.lives}"
        level_text = f"Level: {self.level}"
        
        self.draw_text_with_outline(frame, score_text, config.SCORE_POSITION, config.UI_FONT, config.UI_FONT_SCALE, config.UI_COLOR, config.UI_FONT_THICKNESS)
        self.draw_text_with_outline(frame, lives_text, config.LIVES_POSITION, config.UI_FONT, config.UI_FONT_SCALE, config.UI_COLOR, config.UI_FONT_THICKNESS)
        self.draw_text_with_outline(frame, level_text, config.LEVEL_POSITION, config.UI_FONT, config.UI_FONT_SCALE, config.UI_COLOR, config.UI_FONT_THICKNESS)

        # --- REMOVED DEBUG TEXT ---
        # The "Hits:" counter is no longer drawn here.

        effect_text = ""
        for effect, end_time in self.active_effects.items():
            remaining_time = max(0, (end_time - time.time()))
            effect_text += f"{effect.replace('_', ' ').title()}: {int(remaining_time) + 1}s  "
        self.draw_text_with_outline(frame, effect_text, config.POWERUP_TIMER_POSITION, config.UI_FONT, 1.0, config.UI_COLOR, config.UI_FONT_THICKNESS)
        
    def check_collisions(self):
        b = self.ball
        p = self.paddle
        if b.x - b.radius <= 0 or b.x + b.radius >= config.SCREEN_WIDTH: b.vx *= -1 #left/right walls, so reverse x velocity
        if b.y - b.radius <= 0: b.vy *= -1  #top wall, so reverse y velocity
        if (b.x > p.x and b.x < p.x + p.w and b.y + b.radius > p.y and b.y - b.radius < p.y + p.h and b.vy > 0):  #checks for collision with paddle
            b.vy *= -1
            paddle_center_x = p.x + p.w / 2
            offset = b.x - paddle_center_x  #negative if left, positive if right
            normalized_offset = offset / (p.w / 2)   #converts offset to -1 to 1 range
            b.vx = normalized_offset * config.PADDLE_BOUNCE_INTENSITY
            self.score += config.POINTS_PER_HIT
            self.hits_this_level += 1
            if self.hits_this_level >= config.HITS_PER_LEVEL:
                self.level_up()
            if random.random() < config.POWERUP_SPAWN_CHANCE:
                self.spawn_powerup()
        if b.y - b.radius > config.SCREEN_HEIGHT:   #gone past bottom edge
            self.lives -= 1
            self.deactivate_all_effects()
            if self.lives <= 0: self.game_state = "game_over"
            else: self.ball.reset(level=self.level)
            
    def spawn_powerup(self):
        max_for_level = 0
        for level_thresh, max_count in config.POWERUP_MAX_ON_SCREEN.items():
            if self.level >= level_thresh: max_for_level = max_count
        if len(self.powerups) < max_for_level:
            available_pool = [ptype for ptype, unlock_level in config.POWERUP_UNLOCK_LEVELS.items() if self.level >= unlock_level]
            if available_pool:
                self.powerups.append(PowerUp(random.choice(available_pool)))
    def update_powerups(self):
        for p in self.powerups[:]:
            p.move()
            if (p.x < self.paddle.x + self.paddle.w and p.x + p.w > self.paddle.x and
                p.y < self.paddle.y + self.paddle.h and p.y + p.h > self.paddle.y):
                self.activate_powerup(p.type)
                self.powerups.remove(p)
            elif p.y > config.SCREEN_HEIGHT:
                self.powerups.remove(p)
    def activate_powerup(self, effect_type):
        opposites = {"wide_paddle": "shrink_paddle", "shrink_paddle": "wide_paddle",
                     "slow_ball": "fast_ball", "fast_ball": "slow_ball"}
        if effect_type in opposites and opposites[effect_type] in self.active_effects:
            self.deactivate_effect(opposites[effect_type])
        if effect_type == "extra_life":
            self.lives += 1
        else:
            self.active_effects[effect_type] = time.time() + config.POWERUP_DURATION_SECONDS
            if effect_type == "wide_paddle": self.paddle.w = self.paddle.base_w * config.PADDLE_WIDE_FACTOR
            elif effect_type == "shrink_paddle": self.paddle.w = self.paddle.base_w * config.PADDLE_SHRINK_FACTOR
            elif effect_type == "slow_ball": self.ball.speed_multiplier = config.BALL_SLOW_FACTOR
            elif effect_type == "fast_ball": self.ball.speed_multiplier = config.BALL_FAST_FACTOR
    def update_effects(self):
        current_time = time.time()
        expired_effects = [effect for effect, end_time in self.active_effects.items() if current_time > end_time]
        for effect in expired_effects:
            self.deactivate_effect(effect)
    def deactivate_effect(self, effect_type):
        if effect_type in self.active_effects: del self.active_effects[effect_type]
        if effect_type == "wide_paddle" or effect_type == "shrink_paddle": self.paddle.w = self.paddle.base_w
        if effect_type == "slow_ball" or effect_type == "fast_ball": self.ball.speed_multiplier = 1.0
    def deactivate_all_effects(self):
        self.paddle.w = self.paddle.base_w
        self.ball.speed_multiplier = 1.0
        self.active_effects = {}

game_objects.py


In [ ]:
import cv2
import config
import random
import numpy as np

class Paddle:
    def __init__(self):
        self.base_w = config.PADDLE_WIDTH    #to retain shape after power-up
        self.w = self.base_w
        self.h = config.PADDLE_HEIGHT
        self.x = (config.SCREEN_WIDTH - self.w) // 2
        self.y = config.SCREEN_HEIGHT - self.h - 40
    def update(self, face_center_x):
        control_zone_width = config.SCREEN_WIDTH * config.FACE_CONTROL_ZONE_PERCENT  # 50% of screen
        zone_margin = (config.SCREEN_WIDTH - control_zone_width) / 2
        input_min = zone_margin
        input_max = config.SCREEN_WIDTH - zone_margin
        clamped_face_x = max(input_min, min(face_center_x, input_max))  #position stops at edges of control zone
        input_range = input_max - input_min
        percent_in_zone = (clamped_face_x - input_min) / input_range  #converts head position to 0-1 range
        output_max = config.SCREEN_WIDTH - self.w
        output_min = 0
        output_range = output_max - output_min
        target_x = output_min + (percent_in_zone * output_range)   #maps 0-1 range to paddle position
        move_x = (target_x - self.x) * config.PADDLE_SMOOTHING     #glides to target
        self.x += int(move_x)
        if self.x < 0: self.x = 0
        if self.x + self.w > config.SCREEN_WIDTH: self.x = config.SCREEN_WIDTH - self.w
    def draw(self, frame):
        x, y, w, h = int(self.x), int(self.y), int(self.w), int(self.h)
        top_color = config.PADDLE_COLOR_TOP
        bottom_color = config.PADDLE_COLOR_BOTTOM
        for i in range(h):        #vertical gradient to give 3D effect
            inter_color = [bottom_color[j] + (top_color[j] - bottom_color[j]) * (i / h) for j in range(3)]
            cv2.line(frame, (x, y + i), (x + w, y + i), inter_color, 1)
        cv2.line(frame, (x, y), (x + w, y), config.PADDLE_HIGHLIGHT_COLOR, 2)
        cv2.line(frame, (x, y), (x, y + h), config.PADDLE_HIGHLIGHT_COLOR, 1)

class Ball:
    def __init__(self):
        self.radius = config.BALL_RADIUS
        self.speed_multiplier = 1.0   
        self.image = cv2.imread(config.BALL_IMAGE_PATH, cv2.IMREAD_UNCHANGED)
        if self.image is not None:
            self.image = cv2.resize(self.image, (self.radius * 2, self.radius * 2))
            self.has_alpha = self.image.shape[2] == 4
        else:
            self.has_alpha = False
        self.reset()
    def reset(self, level=1):
        self.x = config.SCREEN_WIDTH // 2   #centered
        self.y = config.SCREEN_HEIGHT // 2
        total_speed_increase = (level - 1) * config.BALL_SPEED_INCREASE  #increases speed with level
        speed_magnitude_x = abs(config.BALL_INITIAL_VX) + total_speed_increase
        speed_magnitude_y = abs(config.BALL_INITIAL_VY) + total_speed_increase
        self.vy = -speed_magnitude_y   #always starts going up
        self.vx = random.uniform(speed_magnitude_x * 0.3, speed_magnitude_x) * random.choice([-1, 1]) #random left/right
    def move(self):    #updates position based on velocity
        self.x += self.vx * self.speed_multiplier
        self.y += self.vy * self.speed_multiplier
    def draw(self, frame):
        if self.image is not None and self.has_alpha:
            x, y, r = int(self.x), int(self.y), self.radius
            x1, y1 = x - r, y - r
            x2, y2 = x + r, y + r
            if x2 < 0 or x1 > config.SCREEN_WIDTH or y2 < 0 or y1 > config.SCREEN_HEIGHT: return
            img_x1, img_x2 = max(0, -x1), (r * 2) - max(0, x2 - config.SCREEN_WIDTH)
            img_y1, img_y2 = max(0, -y1), (r * 2) - max(0, y2 - config.SCREEN_HEIGHT)
            frame_x1, frame_x2 = max(x1, 0), min(x2, config.SCREEN_WIDTH)
            frame_y1, frame_y2 = max(y1, 0), min(y2, config.SCREEN_HEIGHT)
            if img_x1 >= img_x2 or img_y1 >= img_y2: return
            ball_slice = self.image[img_y1:img_y2, img_x1:img_x2]
            roi = frame[frame_y1:frame_y2, frame_x1:frame_x2]
            if roi.shape[:2] == ball_slice.shape[:2]:
                alpha_s = ball_slice[:, :, 3] / 255.0
                alpha_l = 1.0 - alpha_s
                for c in range(3):
                    roi[:, :, c] = (alpha_s * ball_slice[:, :, c] + alpha_l * roi[:, :, c])
        else:
            cv2.circle(frame, (int(self.x), int(self.y)), self.radius, (0,0,255), -1)

class PowerUp:
    def __init__(self, type_name):
        self.w = config.POWERUP_SIZE
        self.h = config.POWERUP_SIZE
        self.type = type_name
        self.image = cv2.imread(config.POWERUP_TYPES[self.type]["image"], cv2.IMREAD_UNCHANGED)
        if self.image is not None:
            self.image = cv2.resize(self.image, (self.w, self.h))
            self.has_alpha = self.image.shape[2] == 4
        else:
            self.has_alpha = False
        self.x = random.randint(0, config.SCREEN_WIDTH - self.w)
        self.y = -self.h
        self.vy = config.POWERUP_SPEED_Y
        self.vx = random.uniform(-config.POWERUP_SPEED_X_MAX, config.POWERUP_SPEED_X_MAX)
    def move(self):
        self.y += self.vy
        self.x += self.vx
        if self.x <= 0 or self.x + self.w >= config.SCREEN_WIDTH: self.vx *= -1
    def draw(self, frame):
        if self.image is None or not self.has_alpha: return
        try:
            y1, y2 = int(self.y), int(self.y + self.h)
            x1, x2 = int(self.x), int(self.x + self.w)
            if y2 < 0 or y1 > config.SCREEN_HEIGHT or x2 < 0 or x1 > config.SCREEN_WIDTH: return
            img_y1, img_y2 = max(0, -y1), self.h - max(0, y2 - config.SCREEN_HEIGHT)
            img_x1, img_x2 = max(0, -x1), self.w - max(0, x2 - config.SCREEN_WIDTH)
            roi_y1, roi_y2 = max(y1, 0), min(y2, config.SCREEN_HEIGHT)
            roi_x1, roi_x2 = max(x1, 0), min(x2, config.SCREEN_WIDTH)
            if img_y1 >= img_y2 or img_x1 >= img_x2: return
            powerup_slice = self.image[img_y1:img_y2, img_x1:img_x2]
            roi = frame[roi_y1:roi_y2, roi_x1:roi_x2]
            if roi.shape[:2] == powerup_slice.shape[:2]:
                alpha_s = powerup_slice[:, :, 3] / 255.0
                alpha_l = 1.0 - alpha_s
                for c in range(3):
                    roi[:, :, c] = (alpha_s * powerup_slice[:, :, c] + alpha_l * roi[:, :, c])
        except Exception: pass